In [52]:
using CSV, Random, StatsBase, LinearAlgebra, Distributions, DataFrames, Plots

In [53]:
# Read data
train = CSV.read("data/housing_train.csv")
test = CSV.read("data/housing_test.csv")

#From DataFrames to Matrices
#input data
X = convert(Matrix, train[:,1:13])
X_test = convert(Matrix, test[:,1:13])

#demand
Y = train[:, 14]
Y_test = test[:,14]

n, p = size(X)

(354, 13)

In [54]:
#add Outliers to train data
percent_outliers = 5
idx_outliers = shuffle(1:n)[1:Int(round(percent_outliers/100*n))]

σ = 1.5; μ=0
X[idx_outliers,:] = (μ .+ (σ.*Random.randn((size(idx_outliers)[1],p)))) + X[idx_outliers,:]

18×13 Array{Float64,2}:
  1.27203    -1.47295     1.28473     …   4.45659    -1.73102    -2.16122 
 -1.48316     0.258967   -0.443175        2.28431     0.668575   -0.628678
 -3.12591    -2.6333      0.907485       -1.11286    -1.82937     2.7136  
 -3.60461    -2.95503    -0.847187       -0.735588   -0.397701    3.2896  
  2.52388     0.517058   -1.87484         1.21371    -0.667525    1.51315 
 -0.797983    6.63083     0.564393    …   0.545131   -0.804266   -1.95628 
 -1.07108     3.89539    -3.16024        -1.29292    -0.889224   -1.29641 
  0.863789   -0.731209   -0.680703        3.70939    -6.92431     0.529326
  1.23695    -0.661774    2.01435         0.287445   -2.14693    -0.586912
  1.68957    -1.07031     1.94114        -0.921036   -0.259276    1.5927  
 -0.48521    -0.116656    0.746018    …  -1.64007    -0.48752     0.429787
 -0.999055   -0.210989   -0.104619       -2.00987     0.0766653  -0.939519
  1.73845     0.0697995  -2.89245        -2.59572     0.0634938  -2.15936 
 

## EVaR Regression Regularized
$$
\min_{s, w} \lambda ||w||^2 + s \log \left(\frac{1}{n\alpha} \sum_{i=1}^n \exp \left(\frac{(Y_i - w^\top X_i)^2}{s}\right) \right)
$$

### $\ell_2$ EVaR

In [57]:
function objective_ℓ2(X,Y,s,w,α, λ)
    """
    α: fraction of data that constitutes the training set ∈ [0,1]
    """
    n = size(X)[1]
    return λ*dot(w, w) + s*log(sum(exp(1/s*(dot(w,X[i,:])-Y[i])^2) for i=1:n)/(n*α))
end

function ∇sobjective_ℓ2(X,Y,s,w,α,λ) # Not affected bY λ but still putting it for function homogeneity issue
    n = size(X)[1]
    return log(sum(exp(1/s*(dot(w,X[i,:])-Y[i])^2) for i=1:n)/(n*α)) - sum(((dot(w,X[i,:])-Y[i])^2)*exp(1/s*(dot(w,X[i,:])-Y[i])^2) for i=1:n)/sum(exp(1/s*(dot(w,X[i,:])-Y[i])^2) for i=1:n)/s
end

function ∇wobjective_ℓ2(X,Y,s,w,α,λ)
    n = size(X)[1]
    return 2*λ*w + 2*sum(((dot(w,X[i,:])-Y[i]))*exp(1/s*(dot(w,X[i,:])-Y[i])^2)*X[i,:] for i=1:n)/sum(exp(1/s*(dot(w,X[i,:])-Y[i])^2) for i=1:n)
end

∇wobjective_ℓ2 (generic function with 1 method)

In [58]:
function gradient_descent_EVaR_ℓ2(X, Y, s_0, w_0, α, λ, c_s, c_w, ε)
    """
    c_s: constant learning rate associated to the ∇ w.r.t s
    c_w: constant learning rate associated to the ∇ w.r.t w
    ε: 
    """
    #println(objective_ℓ2(X,Y,s_0,w_0,α,λ))
    list_f = [objective_ℓ2(X,Y,s_0,w_0,α,λ)]
    s=s_0; w=w_0
    n_grad = dot(∇wobjective_ℓ2(X,Y,s,w,α,λ), ∇wobjective_ℓ2(X,Y,s,w,α,λ)) + dot(∇sobjective_ℓ2(X,Y,s,w,α,λ),∇sobjective_ℓ2(X,Y,s,w,α,λ))
    k=1
    while n_grad > ε
        if k == 30000
            break
        end
        s = s - c_s*∇sobjective_ℓ2(X,Y,s,w,α,λ)
        s = max(0.000000001,s)
        w = w - c_w*∇wobjective_ℓ2(X,Y,s,w,α,λ)
        n_grad = dot(∇wobjective_ℓ2(X,Y,s,w,α,λ), ∇wobjective_ℓ2(X,Y,s,w,α,λ)) + dot(∇sobjective_ℓ2(X,Y,s,w,α,λ),∇sobjective_ℓ2(X,Y,s,w,α,λ))
        push!(list_f, objective_ℓ2(X,Y,s,w,α,λ))
        k = k+1
    end
    list_f, s, w
end

gradient_descent_EVaR_ℓ2 (generic function with 1 method)

In [59]:
# To be implemented
    function nesterov_gradient_descent_EVaR_ℓ2(X, Y, s_0, w_0, α, λ, c_s, c_w, ε, μ)
            """
        c_s: constant learning rate associated to the ∇ w.r.t s
        c_w: constant learning rate associated to the ∇ w.r.t w
        ε: stopping gradient norm 
        μ: momentum
        """
        #println(objective_ℓ2(X,Y,s_0,w_0,α,λ))
        list_f = [objective_ℓ2(X,Y,s_0,w_0,α,λ)]
        s=s_0; w=w_0
        n_grad = dot(∇wobjective_ℓ2(X,Y,s,w,α,λ), ∇wobjective_ℓ2(X,Y,s,w,α,λ)) + dot(∇sobjective_ℓ2(X,Y,s,w,α,λ),∇sobjective_ℓ2(X,Y,s,w,α,λ))
        k=1
        while n_grad > ε
            if k == 30000
                break
            end
            s = (s - c_s*∇sobjective_ℓ2(X,Y,s,w,α,λ))*(1+μ) - μ*s
            s = max(0.000000001,s)
            w = (w - c_w*∇wobjective_ℓ2(X,Y,s,w,α,λ))*(1+μ) - μ*w
            n_grad = dot(∇wobjective_ℓ2(X,Y,s,w,α,λ), ∇wobjective_ℓ2(X,Y,s,w,α,λ)) + dot(∇sobjective_ℓ2(X,Y,s,w,α,λ),∇sobjective_ℓ2(X,Y,s,w,α,λ))
            push!(list_f, objective_ℓ2(X,Y,s,w,α,λ))
            k = k+1
        end
        list_f, s, w
    end

nesterov_gradient_descent_EVaR_ℓ2 (generic function with 1 method)

### $\ell_1$ EVaR

In [60]:
function objective_ℓ1(X,Y,s,w,α, λ)
    """
    α: fraction of data that constitutes the training set ∈ [0,1]
    """
    n = size(X)[1]
    return λ*dot(w, w) + s*log(sum(exp(1/s*abs(dot(w,X[i,:])-Y[i])) for i=1:n)/(n*α))
end

function ∇sobjective_ℓ1(X,Y,s,w,α,λ) # Not affected bY λ but still putting it for function homogeneity issue
    n = size(X)[1]
    return log(sum(exp(1/s*abs(dot(w,X[i,:])-Y[i])) for i=1:n)/(n*α)) - sum(abs(dot(w,X[i,:])-Y[i])*exp(1/s*(abs(dot(w,X[i,:])-Y[i]))) for i=1:n)/sum(exp(1/s*abs(dot(w,X[i,:])-Y[i])) for i=1:n)/s
end

function ∇wobjective_ℓ1(X,Y,s,w,α,λ)
    n = size(X)[1]
    return 2*λ*w + sum(sign.((dot(w,X[i,:])-Y[i]))*exp(1/s*(abs(dot(w,X[i,:])-Y[i])))*X[i,:] for i=1:n)/sum(exp(1/s*(abs(dot(w,X[i,:])-Y[i]))) for i=1:n)
end

∇wobjective_ℓ1 (generic function with 1 method)

In [61]:
function gradient_descent_EVaR_ℓ1(X, Y, s_0, w_0, α, λ, c_s, c_w, ε)
    """
    c_s: constant learning rate associated to the ∇ w.r.t s
    c_w: constant learning rate associated to the ∇ w.r.t w
    ε: 
    """
    #println(objective_ℓ1(X,Y,s_0,w_0,α,λ))
    list_f = [objective_ℓ1(X,Y,s_0,w_0,α,λ)]
    s=s_0; w=w_0
    n_grad = dot(∇wobjective_ℓ1(X,Y,s,w,α,λ), ∇wobjective_ℓ1(X,Y,s,w,α,λ)) + dot(∇sobjective_ℓ1(X,Y,s,w,α,λ),∇sobjective_ℓ1(X,Y,s,w,α,λ))
    k=1
    while n_grad > ε
        s = s - c_s*∇sobjective_ℓ1(X,Y,s,w,α,λ)
        s = max(0.000000001,s)
        w = w - c_w*∇wobjective_ℓ1(X,Y,s,w,α,λ)
        n_grad = dot(∇wobjective_ℓ1(X,Y,s,w,α,λ), ∇wobjective_ℓ1(X,Y,s,w,α,λ)) + dot(∇sobjective_ℓ1(X,Y,s,w,α,λ),∇sobjective_ℓ1(X,Y,s,w,α,λ))
        push!(list_f, objective_ℓ1(X,Y,s,w,α,λ))
        if k == 30000
            break
        end
        k += 1
    end
    list_f, s, w
end

gradient_descent_EVaR_ℓ1 (generic function with 1 method)

In [62]:
# To be implemented
function nesterov_gradient_descent_EVaR_ℓ1()
    
end

nesterov_gradient_descent_EVaR_ℓ1 (generic function with 1 method)

## Validation & Hyperparameter tuning Functions

In [1]:
function fit_least_squares(X_i, Y_i, w_opt)
    return (Y_i - dot(w_opt, X_i))^2
end

function fit_least_absolute_values(X_i, Y_i, w_opt)
    return abs(Y_i - dot(w_opt, X_i))
end

function calculate_MSE(X, Y, w_opt)
    return mean((X*w_opt-Y).^2)
end

function calculate_MAE(X, Y, w_opt)
    return mean(broadcast(abs, X*w_opt-Y))
end

calculate_MAE (generic function with 1 method)

In [2]:
function get_training_and_validation_indices(X, Y, w_opt, train_is_worse, k)
    """
    train_is_worse: boolean, true if the training set contains the worst errors, false if it's the validation
    k: number of observations in the training set
    """
    n,p = size(X)
    least_squares_df = DataFrame()
    least_squares_df.Obs_Index = 1:n
    least_squares_df.LS_Value = [fit_least_squares(X[i,:], Y[i], w_opt) for i=1:n]
    least_squares_sorted = sort!(least_squares_df, :LS_Value, rev=true)
    if train_is_worse == true
        train_indices = least_squares_sorted[1:k, :Obs_Index]
        val_indices = least_squares_sorted[k+1:n, :Obs_Index]
    else
        val_indices = least_squares_sorted[1:k, :Obs_Index]
        train_indices = least_squares_sorted[k+1:n, :Obs_Index]
    end
    
    return train_indices, val_indices
end

get_training_and_validation_indices (generic function with 1 method)

## Performing the Regressions (Housing Dataset) - 70/30 Train/Val splits

### EVaRegression ℓ2 with gradient descent

In [67]:
for train_prop in [0.7,0.6,0.5]

    #### Finding best EVaRegression l2 with random start

    n, p = size(X);   k = floor(Int, train_prop*n);   α = k/n;   ε=0.001
    # Initial values, we use RLS weights as warm start
    w_0 = ones(p)./100
    s_0 = 0.9


    best_λ_EVaR = 0
    best_c_s_EVaR = 0
    best_c_w_EVaR = 0
    best_MSE_EVaR = 1000000
    for λ_EVaR in [0, 0.01, 0.1, 0.2, 1, 10, 20, 50, 100]
        for c_s_EVaR in [0.0001, 0.001, 0.01, 0.1]
            for c_w_EVaR in [0.0001, 0.001, 0.01, 0.1]
                list_f_opt, s_opt, w_opt_EVaR = gradient_descent_EVaR_ℓ2(X,Y,s_0,w_0,α,λ_EVaR,c_s_EVaR,c_w_EVaR,ε)
                train_indices_EVaR, val_indices_EVaR = get_training_and_validation_indices(
                    X, Y, w_opt_EVaR, true, k
                )
                X_train_EVaR, Y_train_EVaR = X[train_indices_EVaR, :], Y[train_indices_EVaR]
                X_val_EVaR, Y_val_EVaR = X[val_indices_EVaR, :], Y[val_indices_EVaR]
                MSE_EVaR = calculate_MSE(X_val_EVaR, Y_val_EVaR, w_opt_EVaR)
                if MSE_EVaR < best_MSE_EVaR
                    best_λ_EVaR = λ_EVaR
                    best_c_s_EVaR = c_s_EVaR
                    best_c_w_EVaR = c_w_EVaR
                    best_MSE_EVaR = MSE_EVaR
                    println("λ=", λ_EVaR, "\t c_s_EVaR=", c_s_EVaR, "\t c_w_EVaR=", c_w_EVaR, "\t MSE=", MSE_EVaR)
                end
            end
        end
    end
    println("Best λ_EVaR = ", best_λ_EVaR, "\t c_s_EVaR=", best_c_s_EVaR, "\t c_w_EVaR=", best_c_w_EVaR)

    # Fitting Stable regression with best λ
    list_f_opt, s_opt, w_opt_EVaR = gradient_descent_EVaR_ℓ2(
        X, Y, s_0, w_0, α, best_λ_EVaR, best_c_s_EVaR, best_c_w_EVaR, ε
    )
    train_indices_EVaR, val_indices_EVaR = get_training_and_validation_indices(
        X, Y, w_opt_EVaR, true, k
    )
    X_train_EVaR, Y_train_EVaR = X[train_indices_EVaR, :], Y[train_indices_EVaR]
    X_val_EVaR, Y_val_EVaR = X[val_indices_EVaR, :], Y[val_indices_EVaR]
    println("Fitted EVaRegression ℓ_2 with best possible λ_EVaR=", best_λ_EVaR, 
        "  c_s_EVaR=", best_c_s_EVaR, "  c_w_EVaR=", best_c_w_EVaR)

        #### Finding best EVaRegression l1 with random warm start

    n, p = size(X);   k = floor(Int, train_prop*n);   α = k/n;   ε=0.001
    # Initial values, we use RLS weights as warm start
    w_0 = ones(p)./100
    s_0 = 0.9


    best_λ_EVaR_ℓ1 = 0
    best_c_s_EVaR_ℓ1 = 0
    best_c_w_EVaR_ℓ1 = 0
    best_MSE_EVaR_ℓ1 = 1000000
    for λ_EVaR_ℓ1 in [0, 0.01, 0.1, 0.2, 1, 10, 20, 50, 100]
        for c_s_EVaR_ℓ1 in [0.0001, 0.001, 0.01, 0.1]
            for c_w_EVaR_ℓ1 in [0.0001, 0.001, 0.01, 0.1]
                list_f_opt, s_opt, w_opt_EVaR_ℓ1 = gradient_descent_EVaR_ℓ1(X,Y,s_0,w_0,α,λ_EVaR_ℓ1,c_s_EVaR_ℓ1,c_w_EVaR_ℓ1,ε)
                train_indices_EVaR_ℓ1, val_indices_EVaR_ℓ1 = get_training_and_validation_indices(
                    X, Y, w_opt_EVaR_ℓ1, true, k
                )
                X_train_EVaR_ℓ1, Y_train_EVaR_ℓ1 = X[train_indices_EVaR_ℓ1, :], Y[train_indices_EVaR_ℓ1]
                X_val_EVaR_ℓ1, Y_val_EVaR_ℓ1 = X[val_indices_EVaR_ℓ1, :], Y[val_indices_EVaR_ℓ1]
                MSE_EVaR_ℓ1 = calculate_MSE(X_val_EVaR_ℓ1, Y_val_EVaR_ℓ1, w_opt_EVaR_ℓ1)
                if MSE_EVaR_ℓ1 < best_MSE_EVaR_ℓ1
                    best_λ_EVaR_ℓ1 = λ_EVaR_ℓ1
                    best_c_s_EVaR_ℓ1 = c_s_EVaR_ℓ1
                    best_c_w_EVaR_ℓ1 = c_w_EVaR_ℓ1
                    best_MSE_EVaR_ℓ1 = MSE_EVaR_ℓ1
                    println("λ=", λ_EVaR_ℓ1, "\t c_s_EVaR_ℓ1=", c_s_EVaR_ℓ1, "\t c_w_EVaR_ℓ1=", c_w_EVaR_ℓ1, "\t MSE=", MSE_EVaR_ℓ1)
                else
                    println(λ_EVaR_ℓ1, "    ",c_s_EVaR_ℓ1, "    ", c_w_EVaR_ℓ1)
                end
            end
        end
    end
    println("Best λ_EVaR_ℓ1 = ", best_λ_EVaR_ℓ1, "\t c_s_EVaR_ℓ1=", best_c_s_EVaR_ℓ1, "\t c_w_EVaR_ℓ1=", best_c_w_EVaR_ℓ1)

    # Fitting Stable regression with best λ
    list_f_opt, s_opt, w_opt_EVaR_ℓ1 = gradient_descent_EVaR_ℓ1(
        X, Y, s_0, w_0, α, best_λ_EVaR_ℓ1, best_c_s_EVaR_ℓ1, best_c_w_EVaR_ℓ1, ε
    )
    train_indices_EVaR_ℓ1, val_indices_EVaR_ℓ1 = get_training_and_validation_indices(
        X, Y, w_opt_EVaR_ℓ1, true, k
    )
    X_train_EVaR_ℓ1, Y_train_EVaR_ℓ1 = X[train_indices_EVaR_ℓ1, :], Y[train_indices_EVaR_ℓ1]
    X_val_EVaR_ℓ1, Y_val_EVaR_ℓ1 = X[val_indices_EVaR_ℓ1, :], Y[val_indices_EVaR_ℓ1]
    println("Fitted EVaR_ℓ1egression ℓ_1 with best possible λ_EVaR_ℓ1=", best_λ_EVaR_ℓ1, 
        "  c_s_EVaR_ℓ1=", best_c_s_EVaR_ℓ1, "  c_w_EVaR_ℓ1=", best_c_w_EVaR_ℓ1)
    


    println("For k = ", k,", n = ",n)
    println("########## The training scores are: ##########")
    println("The MSE for the EVaR_ℓ1 Regression is for ε = 0.001 : ", 
        calculate_MSE(X_train_EVaR_ℓ1, Y_train_EVaR_ℓ1, w_opt_EVaR_ℓ1))
    println("The MSE for the EVaR Regression is for ε = 0.001 : ", 
        calculate_MSE(X_train_EVaR, Y_train_EVaR, w_opt_EVaR))


    println("########## The validation scores are: ##########")
    println("The MSE for the EVaR_ℓ1 Regression is for ε = 0.001 : ", 
        calculate_MSE(X_val_EVaR_ℓ1, Y_val_EVaR_ℓ1, w_opt_EVaR_ℓ1))
    println("The MSE for the EVaR Regression is for ε = 0.001 : ", 
        calculate_MSE(X_val_EVaR, Y_val_EVaR, w_opt_EVaR))


    println("########## The test scores are: ##########")
    println("The MSE for the EVaR_ℓ1 Regression is for ε = 0.001 : ", 
        calculate_MSE(X_test, Y_test, w_opt_EVaR_ℓ1))
    println("The MSE for the EVaR Regression is for ε = 0.001 : ", 
        calculate_MSE(X_test, Y_test, w_opt_EVaR))

λ=0.0	 c_s_EVaR=0.0001	 c_w_EVaR=0.0001	 MSE=0.014455699425704602
λ=0.0	 c_s_EVaR=0.001	 c_w_EVaR=0.0001	 MSE=0.013959216578132911
λ=0.0	 c_s_EVaR=0.01	 c_w_EVaR=0.0001	 MSE=0.013942508622192111
λ=0.0	 c_s_EVaR=0.1	 c_w_EVaR=0.0001	 MSE=0.013941329327492747
λ=0.01	 c_s_EVaR=0.001	 c_w_EVaR=0.0001	 MSE=0.01385217527775273
λ=0.01	 c_s_EVaR=0.01	 c_w_EVaR=0.0001	 MSE=0.013828770847216319
λ=0.01	 c_s_EVaR=0.1	 c_w_EVaR=0.0001	 MSE=0.013827220667039185
λ=0.1	 c_s_EVaR=0.0001	 c_w_EVaR=0.0001	 MSE=0.013448002066065437
λ=0.1	 c_s_EVaR=0.001	 c_w_EVaR=0.0001	 MSE=0.012564338553408928
λ=0.1	 c_s_EVaR=0.1	 c_w_EVaR=0.0001	 MSE=0.012563205022912237
λ=0.2	 c_s_EVaR=0.001	 c_w_EVaR=0.0001	 MSE=0.0120642119408125
λ=0.2	 c_s_EVaR=0.01	 c_w_EVaR=0.0001	 MSE=0.012057988636107714
λ=0.2	 c_s_EVaR=0.1	 c_w_EVaR=0.0001	 MSE=0.012055680799439028
λ=1.0	 c_s_EVaR=0.001	 c_w_EVaR=0.0001	 MSE=0.011671703525701209
λ=1.0	 c_s_EVaR=0.01	 c_w_EVaR=0.001	 MSE=0.011670581135417886
λ=1.0	 c_s_EVaR=0.1	 c_w_EVaR=0.01	 